# stuhub

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# import requests
import time
import pandas as pd

#dodatkowe
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException



Sesja bez VPN,  w trybie incognito. 

> strona wykrywa lokalizacje i podaje różne wydarzenai per kraj

In [2]:
chrome_options = Options()
# Run the argument with incognito
chrome_options.add_argument("--incognito") # używać bez VNP

In [3]:
urls = ["https://www.stubhub.pl/bilety-na-sport/category/28/",
"https://www.stubhub.pl/bilety-na-koncerty/category/1/",
"https://www.stubhub.pl/bilety-na-teatr-i-sztuka/category/174/"]

In [4]:
links_of_events_per_cat = dict()

In [5]:
driver = webdriver.Chrome(executable_path='./selenium/chromedriver', chrome_options=chrome_options)
print(time.ctime() )
for web in urls:
    driver.get(web) 


    # Wait 30 seconds for page to load and extract the element after it loads
    timeout = 30
    try:
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "root")))
    except TimeoutException:
        print('Timed out waiting for page to load')
        driver.quit()
     ####################################################


    while True:
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "load-more__link")))
            driver.find_element_by_xpath('//a[contains(text(), "Zobacz więcej wydarzeń")]').click()
        except TimeoutException:
            print('Nie ma już przycisku "Zobacz więcej"')
            break


    cat = " ".join(web.split("/")[3].split("-")[2:])
    
    try:
        events = driver.find_elements_by_css_selector("a.PerformerCardEvent__redirection")
    except:
        events = []
    
    links_of_events_per_cat[cat] = [link.get_attribute("href") for link in events ]
    print(f"ilość eventów w kategorii {cat}: {len(links_of_events_per_cat[cat])}")
    print(time.ctime() )

<ipython-input-5-156a3afc346b>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='./selenium/chromedriver', chrome_options=chrome_options)


Wed Jul 14 20:40:43 2021
Nie ma już przycisku "Zobacz więcej"
ilość eventów w kategorii sport: 134
Wed Jul 14 20:41:06 2021
Nie ma już przycisku "Zobacz więcej"
ilość eventów w kategorii koncerty: 3
Wed Jul 14 20:41:19 2021
Nie ma już przycisku "Zobacz więcej"
ilość eventów w kategorii teatr i sztuka: 1
Wed Jul 14 20:41:30 2021


SESJA Z VPN

In [6]:
chrome_options = Options()
chrome_options.add_extension('./selenium/proxy.zip') # VPN ( nie działa z trybem incognito)

In [7]:
# driver.get(list_of_events[0]) 

In [8]:
# tickets = driver.find_elements_by_css_selector("li.RoyalTicketListPanel")
# print(f"ilość ofert biletowych: {len()}")

In [12]:
for cat in links_of_events_per_cat.keys():
    print(cat)
    for link in links_of_events_per_cat[cat]:
        print(link)

sport
https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-7-30-2021/event/104709350/
https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-7-31-2021/event/104707699/
https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-8-1-2021/event/104709349/
https://www.stubhub.pl/night-of-freestyle-tickets-night-of-freestyle-koln-lanxess-arena-6-11-2021/event/104374369/
https://www.stubhub.pl/biglietti-gran-premio-d-italia-monza-autodromo-nazionale-di-monza-4-9-2021/event/104707687/
https://www.stubhub.pl/biglietti-gran-premio-d-italia-monza-autodromo-nazionale-di-monza-3-9-2021/event/104707688/
https://www.stubhub.pl/biglietti-gran-premio-d-italia-monza-autodromo-nazionale-di-monza-5-9-2021/event/104709426/
https://intl.stubhub.pl/bilety-na-swiss-indoors-basel-2021-atp-world-tour-500-monday/ev1462494?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104825452&utm_source=sh-pl
https://intl.stubhub.pl/bilety-na-swiss-indoors-basel-2021-atp-world-tour-500-sunday/ev1462491?

In [49]:
driver = webdriver.Chrome(executable_path='./selenium/chromedriver', chrome_options=chrome_options)


plik_z_pobranymi_ofertami = '2021-07-13_stuhub_tickets.csv'
columny =  ["link",
            "brak_biletow",
            "ilosc_ofert_biletowych",
            "event_name",
            "event_info",
            "event_data",
            "event_location",  
            "category",
            "ticket_sector_name",
            "ticket_place",
            "ticket_quantity",
            "ticket_price",
            "ticket_price_unit"
            ]


df_offers = pd.DataFrame(list(), columns=columny)

print(time.ctime() )
#kategorie
for cat in links_of_events_per_cat.keys():
    print(cat)
    #eventy w kategorii
    for link in links_of_events_per_cat[cat][:]:
        driver.get(link) 
    
        print(link)

        # Wait 30 seconds for page to load and extract the element after it loads
        timeout = 30
        try:
            WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CLASS_NAME, "RoyalTicketListPanel")))
        except TimeoutException:
            print('Timed out waiting for page to load')
        #     continue  
        ####################################################
        
        i = driver
        
        try:
            event_name = i.find_element_by_css_selector("h1.event-title").text.strip()
        except:
            event_name = ""
        print("event_name",event_name)

        try:
            event_info = i.find_element_by_css_selector("div.event-info").text.replace("Do ustalenia", " ").strip().split(",")
        except:
            event_info = ""
        print("event_info",event_info)


        try:
#             event_data = event_info[1].split(" w ")[0].replace("•","").strip()
            event_data = ",".join(event_info).replace("•",",").split(" w ")[0]
        except:
            event_data = ""
        print("event_data",event_data)



        try:
#             event_location = ", ".join(event_info[1:]).replace("•","").strip().split(" w ")[1]
            event_location = ",".join(event_info).replace("•","").split(" w ")[1]
        except:
            event_location = ""
        print("event_location",event_location)    
        
        # załadowanie wszystkich elementów poprzes scrollowanie
        height_start = 0
        while True:
            print("Start")
            try:
                scr1 = driver.find_element_by_xpath("//ul[@class='RoyalTicketList__container ']")
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scr1)
                time.sleep(5)
                height_end = len(driver.find_elements_by_css_selector("li.RoyalTicketListPanel "))
                print("height_start:",height_start, "  ", "height_end:", height_end)

                if height_start == height_end:
                    print('Nie ma wiecej"')
                    break
                height_start = height_end
                brak_biletow = False
            except:
                brak_biletow = True
                tickets = []
                break
        #########################
        
        
        if brak_biletow:
            print("brak biletow, nast. link prosze \n\n\n\n")
            ticket_sector_name = ""
            ticket_place = ""
            ticket_quantity = ""
            ticket_price = ""
            ticket_price_unit = ""
            
            values = {
                "link":link,
                "brak_biletow" : brak_biletow,
                "ilosc_ofert_biletowych": len(tickets),
                "event_name":event_name,
                "event_info":event_info,
                "event_data":event_data,
                "event_location":event_location,  
                "category":category,
                "ticket_sector_name":ticket_sector_name,
                "ticket_place":ticket_place,
                "ticket_quantity":ticket_quantity,
                "ticket_price":ticket_price,
                "ticket_price_unit":ticket_price_unit,
            }
            
            df_offers = df_offers.append(values, ignore_index = True)

            try:
                df_offers.to_csv(plik_z_pobranymi_ofertami, index=False)
                print("plik z CSV zapisany poprawnie")
            except:
                print("BŁĄD ZAPISU: plik CSV")
            
            continue

        tickets = driver.find_elements_by_css_selector("li.RoyalTicketListPanel")
        print(f"ilość ofert biletowych: {len(tickets)}")
        
        
        for idx, i in enumerate(tickets, start=1):
            print(f"-----oferta {idx}-----")
            category = cat
            print(category)
            
            try:
                ticket_sector_name = i.find_element_by_css_selector("div.SectionRowSeat__sectionTitle").text.strip()
            except:
                ticket_sector_name = ""
            print(ticket_sector_name)

            try:
                ticket_place = i.find_element_by_css_selector("span.SectionRowSeat__row").text.strip()
            except:
                ticket_place = ""
            print(ticket_place)

            try:
                div_full = i.find_element_by_xpath('//*[@class="RoyalTicketListPanel__SecondaryInfo"]').text.strip()
                div_span = i.find_element_by_xpath('//*[@class="RoyalTicketListPanel__SecondaryInfo"]' + "//span").text.strip()
                ticket_quantity = div_full.replace(div_span, '').strip()
            except:
                ticket_quantity = ""
            print(ticket_quantity)

            try:
                ticket_price = i.find_element_by_css_selector("div.AdvisoryPriceDisplay__content").text.strip()
            except:
                ticket_price = ""
            print(ticket_price)

            try:
                ticket_price_unit = i.find_element_by_css_selector("div.PriceDisplay__each").text.strip()
            except:
                ticket_price_unit = ""
            print(ticket_price_unit)
            
            
            values = {
                "link":link,
                "brak_biletow" : brak_biletow,
                "ilosc_ofert_biletowych": len(tickets),
                "event_name":event_name,
                "event_info":event_info,
                "event_data":event_data,
                "event_location":event_location,  
                "category":category,
                "ticket_sector_name":ticket_sector_name,
                "ticket_place":ticket_place,
                "ticket_quantity":ticket_quantity,
                "ticket_price":ticket_price,
                "ticket_price_unit":ticket_price_unit,
            }


            df_offers = df_offers.append(values, ignore_index = True)

            try:
                df_offers.to_csv(plik_z_pobranymi_ofertami, index=False)
                print("plik z CSV zapisany poprawnie")
            except:
                print("BŁĄD ZAPISU: plik CSV")
            
            
            
        print(f"koniec linku: {link}")
        print(time.ctime() )
        print("\n\n\n")
    print(f"koniec kategori: {cat}")
    print(time.ctime() )
    print("\n\n\n")
print("THE END")            

<ipython-input-49-92d31101890a>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='./selenium/chromedriver', chrome_options=chrome_options)


Thu Jul 15 10:40:00 2021
sport
https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-7-30-2021/event/104709350/
event_name F1 Hungary Grand Prix 2021 - Weekend Tickets
event_info ['•   w Hungaroring', ' Mogyoród', ' Budapest']
event_data ,  
event_location Hungaroring, Mogyoród, Budapest
Start
height_start: 0    height_end: 1
Start
height_start: 1    height_end: 1
Nie ma wiecej"
ilość ofert biletowych: 1
-----oferta 1-----
sport
Silver 5
Rząd 5
2 biletów
801 zł
sztuka
plik z CSV zapisany poprawnie
koniec linku: https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-7-30-2021/event/104709350/
Thu Jul 15 10:40:28 2021




https://www.stubhub.pl/f1-hungary-tickets-mogyorod-hungaroring-7-31-2021/event/104707699/
event_name F1 Hungary Grand Prix 2021 - VIP (Paddock Club) 2 Days - Saturday & Sunday Tickets
event_info ['•   w Hungaroring', ' Mogyoród', ' Budapest']
event_data ,  
event_location Hungaroring, Mogyoród, Budapest
Start
height_start: 0    height_end: 1
Start
heigh

plik z CSV zapisany poprawnie
-----oferta 39-----
sport
Tribuna Seconda Variante
Rząd TOP
1 bilet
3193 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 40-----
sport
Tribuna Alta Velocità A

1 bilet
3198 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 41-----
sport
Tribuna Vedano

1 bilet
3198 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 42-----
sport
Gradinate GF11 Interno Parabolica
Rząd TOP
1 bilet
4566 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 43-----
sport
Gradinate GF10 Interno Parabolica
Rząd TOP
1 bilet
4566 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 44-----
sport
Tribuna 21B Laterale Parabolica
Rząd TOP
1 bilet
4571 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 45-----
sport
Tribuna 21A Laterale Parabolica
Rząd TOP
1 bilet
4571 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 46-----
sport
Tribuna 21C Laterale Parabolica
Rząd TOP
1 bilet
4571 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 47-----
sport
Tribuna 21D Laterale Parabol

3317 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 11-----
sport
Tarzan-In Sector 3 Silver I
Rząd G.A.
1 - 6 biletów
3317 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 12-----
sport
Ben Pon Sector 2 Silver I
Rząd G.A.
1 - 6 biletów
3317 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 13-----
sport
Arena Sector 2 Silver I
Rząd G.A.
1 - 6 biletów
3317 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 14-----
sport
Eastside Sector 2B Silver I
Rząd G.A.
1 - 6 biletów
3317 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 15-----
sport
Arena Sector 1 Gold
Rząd G.A.
1 - 6 biletów
4346 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 16-----
sport
Ben Pon Sector 1 Gold
Rząd G.A.
1 - 6 biletów
4346 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 17-----
sport
Eastside Sector 3 Gold
Rząd G.A.
1 - 6 biletów
4346 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 18-----
sport
Tarzan In Sector 1 Gold
Rząd G.A.
1 - 6 biletów
4346 zł
sztuka
plik z CSV zapisany poprawnie


956 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 26-----
sport
Tribune Inf

1 - 5 biletów
956 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 27-----
sport
Tribune Inter

1 - 5 biletów
960 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 28-----
sport
Tribune Inf
Rząd _
1 - 5 biletów
1002 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 29-----
sport
Tribune Sup

1 - 5 biletów
1007 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 30-----
sport
Tribune Inf

1 - 5 biletów
1052 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 31-----
sport
Tribune Inf G7

1 - 5 biletów
2091 zł
sztuka
plik z CSV zapisany poprawnie
koniec linku: https://www.stubhub.pl/billets-tournoi-des-6-nations-saint-denis-stade-de-france-12-2-2022/event/104844646/
Thu Jul 15 10:48:07 2021




https://www.stubhub.pl/billets-tournoi-des-6-nations-saint-denis-stade-de-france-19-3-2022/event/104844793/
event_name Six Nations Rugby 2022 - France v England Tickets
event_info ['sob.', ' 19 mar 2022 •   w

...



1556 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 32-----
sport
Longside Lower Tier N1406


1556 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 33-----
sport
Longside Lower Tier N1406


1556 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 34-----
sport
VIP Packages W205


1601 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 35-----
sport
Longside Upper Tier N3408


2082 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 36-----
sport
Longside Upper Tier N3404


2196 zł
sztuka
plik z CSV zapisany poprawnie
koniec linku: https://www.stubhub.pl/manchester-united-tickets-manchester-united-manchester-old-trafford-28-12-2021/event/104863507/
Thu Jul 15 11:04:31 2021




https://www.stubhub.pl/manchester-united-tickets-manchester-united-manchester-old-trafford-14-8-2021/event/104863509/
event_name Manchester United vs Leeds United Tickets
event_info ['sob.', ' 14 sie 2021 • 12:25 w Old Trafford', ' Manchester', ' LAN']
event_data sob., 14 sie 2021 , 12:25
event_location

Longside Lower Tier N1405


1368 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 29-----
sport
Longside Lower Tier N1406


1368 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 30-----
sport
Longside Lower Tier N1405


1368 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 31-----
sport
VIP Packages W205


1601 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 32-----
sport
Longside Upper Tier N3408


2082 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 33-----
sport
Longside Upper Tier N3404


2196 zł
sztuka
plik z CSV zapisany poprawnie
koniec linku: https://www.stubhub.pl/manchester-united-tickets-manchester-united-manchester-old-trafford-4-12-2021/event/104864279/
Thu Jul 15 11:05:19 2021




https://www.stubhub.pl/giants-live-tickets-leeds-first-direct-arena-9-4-2021/event/104098150/
event_name Resale Giants Live Europe’s Strongest Man 2021 Tickets
event_info ['sob.', ' 04 wrz 2021 • 17:01 w First Direct Arena', ' Leeds', ' WYK']
event_data sob., 04 wrz 2021 , 17:01

https://www.stubhub.pl/athletic-bilbao-fc-bilbao-tickets-11-13-2021/event/104858005/
Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-azerbaijan-grand-prix-2021-friday/ev1416292?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104711024&utm_source=sh-pl
Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-azerbaijan-grand-prix-2021-weekend/ev1416293?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104711023&utm_source=sh-pl
Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-azerbaijan-grand-prix-2021-sunday/ev1416295?afl=1Q606O8YRW&utm_medium=referral&utm_ca

Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-mexico-2021-friday/ev1366584?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104594491&utm_source=sh-pl
Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-mexico-2021-saturday/ev1366585?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104594482&utm_source=sh-pl
Timed out waiting for page to load
event_name 
event_info 
event_data 
event_location 
Start
brak biletow, nast. link prosze 




plik z CSV zapisany poprawnie
https://intl.stubhub.pl/bilety-na-f1-mexico-grand-prix-2021-vip-paddock-club-2-days-saturday-and-sunday/ev1416276?afl=1Q606O8YRW&utm_medium=referral&utm_campaign=104711016&utm_source=sh-pl
Timed out waiting for page to load
event_name 


https://www.stubhub.pl/bilety-na-pearl-jam-krakow-tauron-arena-14-7-2022/event/104490923/
event_name Bilety na Pearl Jam Kraków
event_info ['czw.', ' 14 lip 2022 •   w Tauron Arena', ' Kraków', ' małopolskie']
event_data czw., 14 lip 2022 ,  
event_location Tauron Arena, Kraków, małopolskie
Start
height_start: 0    height_end: 25
Start
height_start: 25    height_end: 25
Nie ma wiecej"
ilość ofert biletowych: 25
-----oferta 1-----
koncerty
Upper Tier C15

1 - 4 biletów
699 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 2-----
koncerty
Upper Tier C22

1 - 4 biletów
699 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 3-----
koncerty
Upper Tier C14

1 - 4 biletów
799 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 4-----
koncerty
Lower Tier A18

1 - 4 biletów
799 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 5-----
koncerty
Upper Tier C9

1 - 4 biletów
799 zł
sztuka
plik z CSV zapisany poprawnie
-----oferta 6-----
koncerty
Upper Tier C8

1 - 4 biletów
799 zł
sztuka
plik 

__Sprawdzenie pobranych danych__

In [119]:
df = pd.read_csv(plik_z_pobranymi_ofertami)
df.sample(4)

,link,brak_biletow,ilosc_ofert_biletowych,event_name,event_info,event_data,event_location,category,ticket_sector_name,ticket_place,ticket_quantity,ticket_price,ticket_price_unit
513,https://www.stubhub.pl/manchester-city-tickets...,False,7,Manchester City vs Arsenal FC Tickets,"['sob.', ' 28 sie 2021 • 12:25 w Etihad Stadiu...","sob., 28 sie 2021 , 12:25","Etihad Stadium, Manchester, Lancashire",sport,VIP Packages,NaN,1 - 6 biletów,1144 zł,sztuka
332,https://www.stubhub.pl/tottenham-hotspur-ticke...,False,19,Tottenham Hotspur vs Manchester United Tickets,"['sob.', ' 30 paź 2021 • 15:00 w Tottenham Hot...","sob., 30 paź 2021 , 15:00","Tottenham Hotspur Stadium, London, England",sport,Longside Upper Tier,Rząd row,1 bilet,1258 zł,sztuka
692,https://intl.stubhub.pl/bilety-na-geelong-cats...,True,0,NaN,NaN,NaN,NaN,sport,NaN,NaN,NaN,NaN,NaN
322,https://www.stubhub.pl/tottenham-hotspur-ticke...,False,18,Tottenham Hotspur vs Watford FC Tickets,"['niedz.', ' 29 sie 2021 • 14:00 w Tottenham H...","niedz., 29 sie 2021 , 14:00","Tottenham Hotspur Stadium, London, England",sport,Longside Lower Tier,Rząd UU,1 bilet,778 zł,sztuka


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   link                    751 non-null    object
 1   brak_biletow            751 non-null    bool  
 2   ilosc_ofert_biletowych  751 non-null    int64 
 3   event_name              694 non-null    object
 4   event_info              694 non-null    object
 5   event_data              694 non-null    object
 6   event_location          694 non-null    object
 7   category                751 non-null    object
 8   ticket_sector_name      688 non-null    object
 9   ticket_place            312 non-null    object
 10  ticket_quantity         513 non-null    object
 11  ticket_price            688 non-null    object
 12  ticket_price_unit       688 non-null    object
dtypes: bool(1), int64(1), object(11)
memory usage: 71.3+ KB


In [178]:
for i in ["event_data","event_location", "ticket_price_unit"]:
    print(i)
    print(df[i].unique())
    print("-------\n\n")

event_data
[',  ' 'sob., 06 lis 2021 , 19:00' nan 'wt., 16 lis 2021 , 18:00'
 'sob., 04 wrz 2021 , 09:01' 'pt., 03 wrz 2021 , 09:01'
 'sob., 12 lut 2022 ,  ' 'sob., 19 mar 2022 ,  ' 'niedz., 06 lut 2022 ,  '
 'niedz., 18 lip 2021 , 07:00' 'pt., 16 lip 2021 , 07:00'
 'sob., 28 maj 2022 , 17:00' 'pt., 27 maj 2022 , 21:00'
 'pon., 20 wrz 2021 , 19:00' 'niedz., 26 gru 2021 , 16:00'
 'sob., 04 gru 2021 , 16:00' 'sob., 23 paź 2021 , 15:00'
 'niedz., 15 sie 2021 , 16:30' 'niedz., 29 sie 2021 , 14:00'
 'sob., 30 paź 2021 , 15:00' 'sob., 26 lut 2022 , 16:00'
 'sob., 11 wrz 2021 , 15:00' 'sob., 12 mar 2022 , 16:00'
 'sob., 06 lis 2021 , 16:00' 'niedz., 22 sie 2021 , 16:30'
 'sob., 25 wrz 2021 , 15:00' 'pt., 03 wrz 2021 , 11:00'
 'czw., 02 wrz 2021 , 11:00' 'sob., 04 wrz 2021 , 11:00'
 'sob., 18 wrz 2021 , 15:00' 'sob., 21 sie 2021 , 17:30'
 'sob., 02 paź 2021 , 15:00' 'pt., 13 sie 2021 , 20:00'
 'sob., 27 lis 2021 , 15:30' 'sob., 14 sie 2021 , 15:00'
 'wt., 30 lis 2021 , 20:45' 'niedz., 17 paź 2

__Zakres dat organizowanych wydarzeń__

In [212]:
df["extract_data"] = df["event_data"].str.extract(r'([0-9]{2}\s[a-zźń]{3}\s[0-9]{4})', expand=False)
# df["extract_hour"] = df["event_data"].str.extract(r'([0-9]{2}:[0-9]{2})', expand=False)

def replace_all(dict, str):
    for key in dict:
        str = str.replace(key, dict[key])
    return str

str_month_to_digi = {"sty":"1",
                     "lut":"2",
                     "mar":"3",
                     "kwi":"4",
                     "maj":"5",
                     "cze":"6",
                     "lip":"7",
                     "sie":"8",
                     "wrz":"9",
                     "paź":"10",
                     "lis":"11",
                     "gru":"12",
                     " ":"."}

df["extract_data"] = df["extract_data"].apply(lambda x: replace_all(str_month_to_digi, str(x)))
df["extract_data"]

0             nan
1             nan
2             nan
3      06.11.2021
4             nan
          ...    
746     13.7.2022
747     13.7.2022
748     13.7.2022
749     13.7.2022
750     28.8.2021
Name: extract_data, Length: 751, dtype: object

In [215]:
df["extract_data"] = pd.to_datetime(df["extract_data"], format='%d.%m.%Y', errors='ignore')
df["extract_data"]

0            NaT
1            NaT
2            NaT
3     2021-11-06
4            NaT
         ...    
746   2022-07-13
747   2022-07-13
748   2022-07-13
749   2022-07-13
750   2021-08-28
Name: extract_data, Length: 751, dtype: datetime64[ns]

In [216]:
df["extract_data"].min()

Timestamp('2021-07-16 00:00:00')

In [217]:
df["extract_data"].max()

Timestamp('2022-07-14 00:00:00')

__Najdrozszy bilet w sprzedazy__

In [129]:
df2 = df.dropna(subset=['ticket_price'])
df2[df2.ticket_price.str.endswith("zł")].ticket_price.str.replace("zł","").str.replace(" ","").astype(int).max()

27222

__Ilość biletów wg kategorii wydarzenia__

In [100]:
df.category.value_counts()

sport       698
koncerty     53
Name: category, dtype: int64

__Ilość wydarzeń bez dostępnych biletów w sprzedaży__

In [53]:
df.brak_biletow.value_counts()

False    688
True      63
Name: brak_biletow, dtype: int64

__Wydarzenie z największą ilością ofert z biltami w obiegu wtórnym__

In [88]:
warunek_1 = df["ilosc_ofert_biletowych"] == df.ilosc_ofert_biletowych.max()
warunek_2 = ~df.duplicated(subset="link")
df[warunek_1 & warunek_2]

,link,brak_biletow,ilosc_ofert_biletowych,event_name,event_info,event_data,event_location,category,ticket_sector_name,ticket_place,ticket_quantity,ticket_price,ticket_price_unit
6,https://www.stubhub.pl/biglietti-gran-premio-d...,False,53,Italian Grand Prix 2021 - Sunday Tickets,"['• w Autodromo Nazionale di Monza', ' Monza...",",","Autodromo Nazionale di Monza, Monza, Lombardia",sport,Prato,Rząd TOP,1 bilet,846 zł,sztuka


In [98]:
df.iloc[6].link

'https://www.stubhub.pl/biglietti-gran-premio-d-italia-monza-autodromo-nazionale-di-monza-5-9-2021/event/104709426/'